### Creating a neural network based on the structure bellow

<img src = "structure.png">

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# Não entendi o porque do subsampling (dividir as dimensões por 2)
# entendi, tem o maxpooling

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        # kernel
        # 1 input channel, 6 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 6 input channels, 16 output channels, 5x5 square convolutional
        self.conv2 = nn.Conv2d(6, 16, 5)

        # affine operation: y = Wx + b
        # flatten the output of conv2d -> 16 channels * 5 * 5 (image dimensions)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):

        # Max pooling over a 2,2 window
        # The output of the first convolutional layer
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))

        # The output of the second conv layer (2 or (2,2) will result on a square output)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)

        # Flatten the except the batch dimension
        x = torch.flatten(x, start_dim =1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
# Manipulating the params
params = list(net.parameters())
print(len(params))
print(params[0])

10
Parameter containing:
tensor([[[[ 0.1624,  0.1251, -0.0329, -0.0932, -0.0174],
          [-0.1920,  0.1147, -0.1692, -0.1977, -0.1986],
          [ 0.1099, -0.1647,  0.1586, -0.1915, -0.0501],
          [ 0.1504,  0.0763, -0.0617, -0.0063, -0.0932],
          [ 0.0053,  0.1353,  0.0286,  0.1243,  0.1720]]],


        [[[-0.1516, -0.1937, -0.1318, -0.0321, -0.1498],
          [ 0.1985,  0.0065,  0.0699, -0.0989,  0.0735],
          [ 0.1156, -0.1568, -0.0795,  0.1702,  0.0161],
          [-0.1234, -0.1510, -0.1610, -0.0693, -0.0964],
          [-0.0239,  0.1395, -0.0928, -0.0582, -0.0782]]],


        [[[ 0.0918,  0.0759,  0.1723, -0.0728, -0.1422],
          [-0.1158,  0.0595, -0.0261, -0.0143, -0.1201],
          [-0.0632,  0.1378,  0.1501, -0.1614, -0.0960],
          [-0.0355, -0.1593,  0.1448, -0.0095,  0.1857],
          [ 0.1499,  0.0027, -0.1408, -0.1105, -0.0541]]],


        [[[-0.1835,  0.0477,  0.0063,  0.1634, -0.1938],
          [ 0.0899, -0.1685,  0.0268,  0.0790,  0.1

In [5]:
# Forward propagation on the net
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0300,  0.0107, -0.0071,  0.0997,  0.0558, -0.0009, -0.0400,  0.1366,
          0.1317, -0.1028]], grad_fn=<AddmmBackward>)


In [10]:
# Turning the gradients into 0 and backpropagating random values
net.zero_grad()
out.backward(torch.rand(1, 10))

In [15]:
## LOSS FUNCTION

output = net(input)
target = torch.randn(10) # dummy target

print(f"output shape {output.shape}")
print(f"target shape {target.shape}")

# Reshaping the target
target = target.view(1, -1)
print(f"target shape {target.shape}")

# Loss criterion
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

output shape torch.Size([1, 10])
target shape torch.Size([10])
target shape torch.Size([1, 10])
tensor(0.9187, grad_fn=<MseLossBackward>)


In [22]:
# see the backward functions from loss to the beggining of the net
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [26]:
# Backpropagating the loss

# Cleaning the gradients
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

# applying the backward 
loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0107,  0.0041, -0.0026, -0.0037, -0.0013, -0.0055])


In [30]:
## UPDATING THE WEIGHTS

## weight = weight - learning_rate * gradient

# example of implementation
# learning_rate = 0.01
# for f in net.parameters():
#     f.data.sub_(f.grad.data * learning_rate)


# Using the built in package torch.optim
import torch.optim as optim

# creating the optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# in the training loop
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # updates the weights